# Cloud Workshop Azure Databricks
## 07. Classification Evaluation
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>
V1.3 12/06/2019

## Evaluating a Classification Model

In this exercise, you will create a pipeline for a classification model, and then apply commonly used metrics to evaluate the resulting classifier.

### Prepare the Data

First, import the libraries you will need and prepare the training and test data:

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Load the source data
# File location and type
file_location = "/FileStore/tables/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(data)

# Split the data
splits = data.randomSplit([0.7, 0.3])

train = splits[0]
test = splits[1]

DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay,Late
21,2,WN,10721,13342,26,57,1
13,1,AA,15016,12892,51,27,1
5,5,FL,10397,11433,9,4,0
22,1,US,11278,14100,35,71,1
23,4,WN,12451,10693,9,5,0
5,7,AA,11298,15016,39,42,1
4,6,UA,13930,14307,71,58,1
10,3,9E,14307,11433,68,140,1
29,7,UA,14057,14771,130,125,1
14,7,UA,14771,11292,20,42,1


### Define the Pipeline and Train the Model
Now define a pipeline that creates a feature vector and trains a classification model

In [5]:
monthdayIndexer = StringIndexer(inputCol="DayofMonth", outputCol="DayofMonthIdx")
weekdayIndexer = StringIndexer(inputCol="DayOfWeek", outputCol="DayOfWeekIdx")
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
originIndexer = StringIndexer(inputCol="OriginAirportID", outputCol="OriginAirportIdx")
destIndexer = StringIndexer(inputCol="DestAirportID", outputCol="DestAirportIdx")
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normNums")
featVect = VectorAssembler(inputCols=["DayofMonthIdx", "DayOfWeekIdx", "CarrierIdx", "OriginAirportIdx", "DestAirportIdx", "normNums"], outputCol="features")

In [6]:
lr = LogisticRegression(labelCol="Late", featuresCol="features")

In [7]:
%%time
pipeline = Pipeline(stages=[monthdayIndexer, weekdayIndexer, carrierIndexer, originIndexer, destIndexer, numVect, minMax, featVect, lr])
model = pipeline.fit(train)

CPU times: user 82.7 ms, sys: 28.5 ms, total: 111 ms
Wall time: 9.86 s

### Test the Model
Now you're ready to apply the model to the test data.

In [9]:
prediction = model.transform(test)
predicted = prediction.select("features", col("prediction").cast("Int"), col("Late").alias("trueLabel"))

predicted.show(100, truncate=False)

+---------------------------------------------+----------+---------+
features |prediction|trueLabel|
+---------------------------------------------+----------+---------+
[25.0,2.0,10.0,11.0,12.0,0.4427860696517413] |1 |1 |
[25.0,2.0,10.0,23.0,12.0,0.5472636815920398] |1 |1 |
[25.0,2.0,10.0,13.0,32.0,0.24378109452736318]|0 |0 |
[25.0,2.0,10.0,48.0,15.0,0.22885572139303484]|0 |1 |
[25.0,2.0,10.0,48.0,34.0,0.19900497512437812]|0 |0 |
[25.0,2.0,10.0,12.0,19.0,0.40298507462686567]|1 |0 |
[25.0,2.0,10.0,12.0,22.0,0.2537313432835821] |0 |1 |
[25.0,2.0,10.0,12.0,22.0,0.7014925373134329] |1 |1 |
[25.0,2.0,10.0,12.0,25.0,0.900497512437811] |1 |1 |
[25.0,2.0,10.0,12.0,1.0,0.2537313432835821] |0 |1 |
[25.0,2.0,10.0,16.0,15.0,0.23880597014925373]|0 |0 |
[25.0,2.0,10.0,46.0,32.0,0.22885572139303484]|0 |0 |
[25.0,2.0,10.0,46.0,41.0,0.23880597014925373]|0 |1 |
[25.0,2.0,10.0,54.0,15.0,0.6368159203980099] |1 |1 |
[25.0,2.0,10.0,0.0,16.0,0.5522388059701493] |1 |1 |
[25.0,2.0,2.0,1.0,24.0,0.4427860696517413] |1 |1 |
[25.0,2.0,2.0,34.0,5.0,0.34328358208955223] |1 |0 |
[25.0,2.0,2.0,27.0,2.0,0.2537313432835821] |0 |1 |
[25.0,2.0,2.0,11.0,24.0,0.4626865671641791] |1 |1 |
[25.0,2.0,2.0,3.0,32.0,0.24378109452736318] |0 |0 |
[25.0,2.0,2.0,3.0,9.0,0.25870646766169153] |0 |0 |
[25.0,2.0,2.0,3.0,11.0,0.29850746268656714] |0 |0 |
[25.0,2.0,2.0,3.0,8.0,0.42786069651741293] |1 |1 |
[25.0,2.0,2.0,3.0,2.0,0.6318407960199005] |1 |1 |
[25.0,2.0,2.0,3.0,13.0,0.24875621890547264] |0 |0 |
[25.0,2.0,2.0,3.0,14.0,0.25870646766169153] |0 |0 |
[25.0,2.0,2.0,3.0,14.0,0.42786069651741293] |1 |1 |
[25.0,2.0,2.0,3.0,14.0,0.8905472636815921] |1 |1 |
[25.0,2.0,2.0,3.0,1.0,0.24378109452736318] |0 |0 |
[25.0,2.0,2.0,3.0,39.0,0.3681592039800995] |1 |0 |
[25.0,2.0,2.0,3.0,44.0,0.3034825870646766] |0 |0 |
[25.0,2.0,2.0,3.0,28.0,0.5024875621890548] |1 |1 |
[25.0,2.0,2.0,10.0,5.0,0.208955223880597] |0 |0 |
[25.0,2.0,2.0,51.0,5.0,0.23383084577114427] |0 |0 |
[25.0,2.0,2.0,2.0,5.0,0.5074626865671642] |1 |1 |
[25.0,2.0,2.0,2.0,7.0,0.5870646766169154] |1 |1 |
[25.0,2.0,2.0,2.0,12.0,0.25870646766169153] |0 |1 |
[25.0,2.0,2.0,14.0,24.0,0.31840796019900497] |0 |1 |
[25.0,2.0,2.0,14.0,1.0,0.5671641791044776] |1 |1 |
[25.0,2.0,2.0,21.0,5.0,0.34328358208955223] |1 |1 |
[25.0,2.0,2.0,21.0,7.0,0.24378109452736318] |0 |0 |
[25.0,2.0,2.0,21.0,2.0,0.6268656716417911] |1 |1 |
[25.0,2.0,2.0,21.0,28.0,0.5323383084577115] |1 |1 |
[25.0,2.0,2.0,0.0,32.0,0.21890547263681592] |0 |0 |
[25.0,2.0,2.0,0.0,5.0,0.23880597014925373] |0 |0 |
[25.0,2.0,2.0,0.0,13.0,0.22388059701492538] |0 |1 |
[25.0,2.0,2.0,0.0,13.0,0.736318407960199] |1 |1 |
[25.0,2.0,2.0,0.0,17.0,0.24875621890547264] |0 |0 |
[25.0,2.0,2.0,0.0,28.0,0.6567164179104478] |1 |1 |
[25.0,2.0,2.0,5.0,2.0,0.24875621890547264] |0 |0 |
[25.0,2.0,2.0,28.0,5.0,0.21890547263681592] |0 |0 |
[25.0,2.0,2.0,28.0,5.0,0.22885572139303484] |0 |0 |
[25.0,2.0,2.0,28.0,5.0,0.6268656716417911] |1 |1 |
[25.0,2.0,2.0,28.0,1.0,0.2537313432835821] |0 |0 |
[25.0,2.0,2.0,29.0,5.0,0.23383084577114427] |0 |0 |
[25.0,2.0,2.0,29.0,24.0,0.7213930348258707] |1 |1 |
[25.0,2.0,11.0,2.0,33.0,0.2537313432835821] |0 |0 |
[25.0,2.0,11.0,2.0,33.0,0.7661691542288557] |1 |1 |
[25.0,2.0,11.0,19.0,6.0,0.24875621890547264] |0 |0 |
[25.0,2.0,11.0,40.0,17.0,0.24875621890547264]|0 |0 |
[25.0,2.0,11.0,65.0,17.0,0.25870646766169153]|0 |0 |
[25.0,2.0,7.0,11.0,22.0,0.5572139303482587] |1 |1 |
[25.0,2.0,7.0,11.0,7.0,0.22388059701492538] |0 |1 |
[25.0,2.0,7.0,11.0,26.0,0.38308457711442784] |1 |1 |
[25.0,2.0,7.0,11.0,64.0,0.3333333333333333] |0 |1 |
[25.0,2.0,7.0,11.0,64.0,0.6069651741293532] |1 |1 |
[25.0,2.0,7.0,23.0,10.0,0.4427860696517413] |1 |1 |
[25.0,2.0,7.0,23.0,14.0,0.3333333333333333] |1 |0 |
[25.0,2.0,7.0,4.0,10.0,0.5373134328358209] |1 |1 |
[25.0,2.0,7.0,26.0,22.0,0.582089552238806] |1 |1 |
[25.0,2.0,7.0,26.0,12.0,0.22885572139303484] |0 |0 |
[25.0,2.0,7.0,26.0,2.0,0.25870646766169153] |0 |0 |
[25.0,2.0,7.0,26.0,13.0,0.6766169154228856] |1 |1 |
[25.0,2.0,7.0,12.0,31.0,0.7412935323383084] 

### Compute Confusion Matrix Metrics
Classifiers are typically evaluated by creating a *confusion matrix*, which indicates the number of:
- True Positives
- True Negatives
- False Positives
- False Negatives

From these core measures, other evaluation metrics such as *precision* and *recall* can be calculated.

In [11]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())

metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])


In [12]:
display(metrics)

metric,value
TP,88986.0
FP,7208.0
TN,81021.0
FN,13288.0
Precision,0.9250680915649625
Recall,0.8700745057394841


### View the Raw Prediction and Probability
The prediction is based on a raw prediction score that describes a labelled point in a logistic function. This raw prediction is then converted to a predicted label of 0 or 1 based on a probability vector that indicates the confidence for each possible label value (in this case, 0 and 1). The value with the highest confidence is selected as the prediction.

In [14]:
prediction.select("rawPrediction", "probability", "prediction", col("Late").alias("trueLabel")).show(100, truncate=False)

+------------------------------------------+------------------------------------------+----------+---------+
rawPrediction |probability |prediction|trueLabel|
+------------------------------------------+------------------------------------------+----------+---------+
[-3.0825342538000324,3.0825342538000324] |[0.043833476614834006,0.956166523385166] |1.0 |1 |
[-5.690441439027893,5.690441439027893] |[0.0033667282648677664,0.9966332717351322]|1.0 |1 |
[2.2338749116016468,-2.2338749116016468] |[0.9032505132792287,0.09674948672077127] |0.0 |0 |
[2.6387119399415555,-2.6387119399415555] |[0.9333118392035147,0.06668816079648528] |0.0 |1 |
[3.600568690248358,-3.600568690248358] |[0.9734177256194031,0.026582274380596932] |0.0 |0 |
[-1.9846967794625812,1.9846967794625812] |[0.12081904832425268,0.8791809516757473] |1.0 |0 |
[1.8695050267280013,-1.8695050267280013] |[0.8664009945464457,0.13359900545355427] |0.0 |1 |
[-9.599668462407973,9.599668462407973] |[6.774660492231863E-5,0.9999322533950777] |1.0 |1 |
[-14.665934925545372,14.665934925545372] |[4.2723340686014186E-7,0.9999995727665931]|1.0 |1 |
[1.6514971913778709,-1.6514971913778709] |[0.8390932974591843,0.16090670254081577] |0.0 |1 |
[2.201885963106002,-2.201885963106002] |[0.9004187431916364,0.09958125680836355] |0.0 |0 |
[2.8038222573918103,-2.8038222573918103] |[0.9428820225415786,0.057117977458421426] |0.0 |0 |
[2.6423836648467818,-2.6423836648467818] |[0.9335400072105399,0.0664599927894602] |0.0 |1 |
[-7.776862813485955,7.776862813485955] |[4.191498556744922E-4,0.9995808501443255] |1.0 |1 |
[-5.907131881223695,5.907131881223695] |[0.0027125987216267532,0.9972874012783733]|1.0 |1 |
[-2.592685095710836,2.592685095710836] |[0.06961068146690792,0.930389318533092] |1.0 |1 |
[-0.05358349744827784,0.05358349744827784]|[0.48660732989431116,0.5133926701056888] |1.0 |0 |
[2.169304468150848,-2.169304468150848] |[0.8974589769826385,0.10254102301736141] |0.0 |1 |
[-3.0455650600303237,3.0455650600303237] |[0.0454093291428003,0.9545906708571997] |1.0 |1 |
[2.5991481637764835,-2.5991481637764835] |[0.9308067368008306,0.06919326319916932] |0.0 |0 |
[1.9780718944694264,-1.9780718944694264] |[0.8784754748290705,0.12152452517092935] |0.0 |0 |
[0.9793524575319559,-0.9793524575319559] |[0.7269797108676315,0.2730202891323685] |0.0 |0 |
[-2.365108304697028,2.365108304697028] |[0.08587235208624906,0.9141276479137509] |1.0 |1 |
[-7.652241958260596,7.652241958260596] |[4.747525564046977E-4,0.9995252474435953] |1.0 |1 |
[2.274467718421679,-2.274467718421679] |[0.9067402756818891,0.09325972431811096] |0.0 |0 |
[2.029978521933743,-2.029978521933743] |[0.8839088739993926,0.11609112600060734] |0.0 |0 |
[-2.3028203517398484,2.3028203517398484] |[0.09088964990867156,0.9091103500913285] |1.0 |1 |
[-14.154299623847024,14.154299623847024] |[7.12632243951997E-7,0.9999992873677561] |1.0 |1 |
[2.2773270734977187,-2.2773270734977187] |[0.906981788413832,0.09301821158616799] |0.0 |0 |
[-0.5140640825334675,0.5140640825334675] |[0.37424129401921163,0.6257587059807883] |1.0 |0 |
[1.1945009378060458,-1.1945009378060458] |[0.7675450867207032,0.23245491327929668] |0.0 |0 |
[-4.069010709695759,4.069010709695759] |[0.016806987815263536,0.9831930121847364] |1.0 |1 |
[3.2507019581514482,-3.2507019581514482] |[0.9626983283423466,0.03730167165765334] |0.0 |0 |
[2.846656079732103,-2.846656079732103] |[0.945145573678857,0.05485442632114302] |0.0 |0 |
[-4.440902564986226,4.440902564986226] |[0.011648021334197593,0.9883519786658024] |1.0 |1 |
[-6.459104089846894,6.459104089846894] |[0.0015637491138085682,0.9984362508861915]|1.0 |1 |
[2.0035297629838054,-2.0035297629838054] |[0.8811671825986164,0.11883281740138363] |0.0 |1 |
[0.6671158325839022,-0.6671158325839022] |[0.6608570453542826,0.3391429546457175] |0.0 |1 |
[-5.94341770327194,5.94341770327194] |[0.0026161871207914227,0.9973838128792086]|1.0 |1 |
[-0.1275029009830213,0.1275029009830213] |[0.4681673882775639,0.5318326117224361] |1.0 |1 |
[2.441964969810698,-2.441964969810698] |[0.

Note that the results include rows where the probability for 0 (the first value in the **probability** vector) is only slightly higher than the probability for 1 (the second value in the **probability** vector). The default *discrimination threshold* (the boundary that decides whether a probability is predicted as a 1 or a 0) is set to 0.5; so the prediction with the highest probability is always used, no matter how close to the threshold.

### Review the Area Under ROC
Another way to assess the performance of a classification model is to measure the area under a *received operator characteristic (ROC) curve* for the model. The **spark.ml** library includes a **BinaryClassificationEvaluator** class that you can use to compute this. A ROC curve plots the True Positive and False Positive rates for varying threshold values (the probability value over which a class label is predicted). The area under this curve gives an overall indication of the models accuracy as a value between 0 and 1. A value under 0.5 means that a binary classification model (which predicts one of two possible labels) is no better at predicting the right class than a random 50/50 guess.

In [17]:
evaluator = BinaryClassificationEvaluator(labelCol="Late", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(prediction)

print ("AUC = ", auc)

AUC = 0.947862342306172

### Review the AreaUnderPR

In [19]:
evaluator = BinaryClassificationEvaluator(labelCol="Late", rawPredictionCol="rawPrediction", metricName="areaUnderPR")
areaUnderPR = evaluator.evaluate(prediction)
print ("Area Under PR = ", areaUnderPR)

Area Under PR = 0.9642020136795382

> Fin